In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D,Dropout,Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array , load_img

In [2]:
base_path = r'D:\Projects\X-cencia Projects\Bullying vs No Bullying\dataset'
train_path = r'D:\Projects\X-cencia Projects\Bullying vs No Bullying\dataset\train'
test_path = r'D:\Projects\X-cencia Projects\Bullying vs No Bullying\dataset\test'

In [3]:
image_gen = ImageDataGenerator(rotation_range=30,width_shift_range=0.1,height_shift_range=0.1,rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

In [4]:
IMG_SHAPE = (128,128,3)

In [5]:
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
# Freezing the Base Model
base_model.trainable = False
#Define our Custom Head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')(global_average_layer)
# Define the Model
model = tf.keras.models.Model(inputs=base_model.input,outputs=prediction_layer)

In [6]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 63, 63, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 63, 63, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [8]:
batch_size = 16

train_image_gen = image_gen.flow_from_directory(train_path,target_size=(224,224),batch_size=batch_size,class_mode='binary')

Found 427 images belonging to 2 classes.


In [9]:
test_image_gen = image_gen.flow_from_directory(test_path,target_size=(128,128),batch_size=batch_size,class_mode='binary')

Found 38 images belonging to 2 classes.


In [10]:
train_image_gen.class_indices

{'Bulling': 0, 'NoBulling': 1}

In [11]:
result = model.fit_generator(train_image_gen,epochs=10,steps_per_epoch=20,validation_data=test_image_gen,validation_steps=12)

C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 4/20 [=====>........................] - ETA: 8s - loss: 0.8632 - accuracy: 0.4258

C:\Anaconda\lib\site-packages\PIL\Image.py:2942: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


UnknownError:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001445822FB30>
Traceback (most recent call last):

  File "C:\Anaconda\lib\site-packages\tensorflow\python\ops\script_ops.py", line 249, in __call__
    ret = func(*args)

  File "C:\Anaconda\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "C:\Anaconda\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 891, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 807, in wrapped_generator
    for data in generator_fn():

  File "C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 933, in generator_fn
    yield x[i]

  File "C:\Users\winwi\AppData\Roaming\Python\Python38\site-packages\keras_preprocessing\image\iterator.py", line 65, in __getitem__
    return 

In [ ]:
model.save('BvsNo_10epoch_85_acc_model.h5')

In [ ]:
model.evaluate(test_image_gen)

In [ ]:
def pred_clean(pred):
  if pred > 0.5:
    return 1
  else:
    return 0

def pred_class(pred):
  res = pred_clean(pred)
  labels = ['Bulling','No Bulling']
  return labels[res]

In [ ]:
img_file = '/content/drive/MyDrive/dataset/test/Bulling/image17.jpeg'
image = load_img(img_file,target_size=(128,128))
plt.imshow(image)
image = img_to_array(image)
image = np.expand_dims(image,axis=0)
image = image / 255

In [ ]:
res = model.predict(image)

In [ ]:
pred = pred_class(res)

In [ ]:
pred